Bank Churn Modelling Case Study

Column Description


RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,
NumOfProducts :How many accounts, bank account affiliated products the person has,
HasCrCard,IsActiveMember :Subjective but for the concept,EstimatedSalary,Exited:Did they leave the bank after all?

Problem Statement:

Given a Bank customer, can we build a classifier that can determine whether they will leave or not using Neural networks?

In [0]:
import tensorflow as tf

In [0]:
#!pip install -U tensorflow==2.0

In [2]:
tf.__version__

'2.0.0'

In [0]:
from google.colab import drive

In [4]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1. Read the dataset

In [0]:
import numpy as np
import pandas as pd

In [0]:
churn=pd.read_csv('/content/drive/My Drive/Great Learning Residency 7.1 Neural Networks contd/project 1 neural networks/Churn.csv')

In [7]:
churn.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


2. Drop the columns which are unique for all users like IDs (2.5 points)

In [8]:
churn.shape 

(10000, 14)

In [9]:
churn['RowNumber'].nunique() # All unique values , should be dropped

10000

In [10]:
churn['CustomerId'].nunique() # All unique values , should be dropped

10000

In [11]:
churn['Surname'].nunique() # Not All unique values , could be dropped

2932

In [12]:
churn['Age'].nunique() # shouldn't be dropped

70

In [13]:
churn['Geography'].nunique() # shouldn't be dropped

3

In [14]:
churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [0]:
bank_churn = churn.drop(columns=['RowNumber','CustomerId','Surname'])

In [16]:
print ("Shape of the data: ", bank_churn.shape)
bank_churn.head()

Shape of the data:  (10000, 11)


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [17]:
bank_churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [0]:
# geography and gender are of object type

In [0]:
# Converting the Geography and Gender to categorial.
bank_churn["Geography"] = bank_churn["Geography"].astype('category')
bank_churn["Gender"] = bank_churn["Gender"].astype('category')

In [20]:
bank_churn["Geography"] = bank_churn["Geography"].cat.codes
bank_churn["Gender"] = bank_churn["Gender"].cat.codes

bank_churn.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,608,2,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,850,2,0,43,2,125510.82,1,1,1,79084.10,0


3. Distinguish the feature and target set (2.5 points)

In [0]:
# Excited is the target column

In [0]:
Target_set=pd.DataFrame(bank_churn['Exited'])
Feature_set=bank_churn.drop(['Exited'],axis=1)

In [23]:
Target_set.head()

,Exited
0,1
1,0
2,1
3,0
4,0


In [24]:
Feature_set.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,0,42,2,0.00,1,1,1,101348.88
1,608,2,0,41,1,83807.86,1,0,1,112542.58
2,502,0,0,42,8,159660.80,3,1,0,113931.57
3,699,0,0,39,1,0.00,2,0,0,93826.63
4,850,2,0,43,2,125510.82,1,1,1,79084.10


4. Divide the data set into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(Feature_set,Target_set,test_size=0.2)

In [26]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(8000, 10) (2000, 10) (8000, 1) (2000, 1)


In [27]:
X_train.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
6188,835,1,0,47,5,108289.28,2,1,1,45859.55
6329,727,1,1,34,2,146407.11,1,1,1,72073.72
4992,794,0,0,62,9,123681.32,3,1,0,173586.63
9604,770,0,1,31,3,155047.56,2,1,1,186064.34
4698,732,0,1,62,2,0.00,2,1,1,25438.87


In [28]:
X_test.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
4610,589,1,1,38,8,92219.21,1,1,0,99106.97
1439,778,0,0,65,7,0.00,1,1,1,77867.23
2439,566,0,0,42,6,0.00,1,1,0,180702.12
6919,782,0,1,33,7,191523.09,1,1,1,167058.75
5418,562,1,1,35,3,142296.13,1,0,1,177112.70


In [29]:
y_test.head()

,Exited
4610,0
1439,0
2439,1
6919,0
5418,0


In [30]:
y_train.head()

,Exited
6188,1
6329,0
4992,1
9604,0
4698,0


In [31]:
print ("Unique train labels: ", np.unique(y_train))

Unique train labels:  [0 1]


In [32]:
print ("Unique test labels: ", np.unique(y_test))

Unique test labels:  [0 1]


5. Normalize the train and test data (2.5 points)

In [0]:
from scipy import stats

X_train_std = stats.zscore(X_train) 
X_test_std = stats.zscore(X_test)

In [34]:
X_test_std

array([[-0.62470756,  0.3398494 ,  0.90270817, ...,  0.64998085,
        -0.9990005 , -0.04484389],
       [ 1.31587228, -0.87606975, -1.10777773, ...,  0.64998085,
         1.0010005 , -0.41556032],
       [-0.86086278, -0.87606975, -1.10777773, ...,  0.64998085,
        -0.9990005 ,  1.37931022],
       ...,
       [-1.93896268, -0.87606975,  0.90270817, ..., -1.53850686,
        -0.9990005 , -0.78383006],
       [ 0.52526568,  1.55576856,  0.90270817, ..., -1.53850686,
         1.0010005 , -0.7414628 ],
       [ 0.45339235,  1.55576856, -1.10777773, ...,  0.64998085,
         1.0010005 ,  0.79128986]])

In [35]:
X_train_std

array([[ 1.9106222 ,  0.2983863 , -1.0930633 , ...,  0.64512003,
         0.96269532, -0.93536625],
       [ 0.79099324,  0.2983863 ,  0.9148601 , ...,  0.64512003,
         0.96269532, -0.4798964 ],
       [ 1.48557787, -0.90843394, -1.0930633 , ...,  0.64512003,
        -1.03875025,  1.28388518],
       ...,
       [ 0.13787635, -0.90843394,  0.9148601 , ...,  0.64512003,
         0.96269532, -0.74940183],
       [-1.71780499, -0.90843394,  0.9148601 , ...,  0.64512003,
         0.96269532, -1.51139583],
       [-1.04395422, -0.90843394, -1.0930633 , ...,  0.64512003,
        -1.03875025, -0.79359744]])

In [0]:
y_train_cat = tf.keras.utils.to_categorical(y_train)
y_test_cat = tf.keras.utils.to_categorical(y_test)

In [37]:
y_train_cat

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

6. Initialize & build the model (10 points)




In [0]:
#Clear out tensorflow memory
tf.keras.backend.clear_session()

#Initialize Sequential model
model = tf.keras.models.Sequential()

#Normalize the data
model.add(tf.keras.layers.BatchNormalization())

In [0]:
#Adding 1st dence layer
model.add(tf.keras.layers.Dense(20, activation='sigmoid',input_shape=(10,))) # we have 10 features so 10 input_shape

#Add 2nd hidden layer
model.add(tf.keras.layers.Dense(10, activation='sigmoid')) 

#Adding OUTPUT layer
model.add(tf.keras.layers.Dense(2, activation='softmax')) # outputs are 2

In [0]:
#Comile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [41]:
model.fit(X_train_std, y_train_cat, 
          validation_data=(X_test_std, y_test_cat), 
          epochs=10,
          batch_size=32)

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 1s 140us/sample - loss: 0.5513 - accuracy: 0.7434 - val_loss: 0.4917 - val_accuracy: 0.8050
Epoch 2/10
8000/8000 [==============================] - 0s 56us/sample - loss: 0.5055 - accuracy: 0.7941 - val_loss: 0.4893 - val_accuracy: 0.8050
Epoch 3/10
8000/8000 [==============================] - 0s 54us/sample - loss: 0.5038 - accuracy: 0.7941 - val_loss: 0.4873 - val_accuracy: 0.8050
Epoch 4/10
8000/8000 [==============================] - 0s 57us/sample - loss: 0.5011 - accuracy: 0.7941 - val_loss: 0.4850 - val_accuracy: 0.8050
Epoch 5/10
8000/8000 [==============================] - 0s 54us/sample - loss: 0.4986 - accuracy: 0.7941 - val_loss: 0.4825 - val_accuracy: 0.8050
Epoch 6/10
8000/8000 [==============================] - 0s 53us/sample - loss: 0.4959 - accuracy: 0.7941 - val_loss: 0.4797 - val_accuracy: 0.8050
Epoch 7/10
8000/8000 [==============================] - 0s 52us/sampl

In [0]:
# can optimise this model by using the learning rate parameter

7. Optimize the model (5 points)

In [0]:
#Compile the model
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)
model.compile(optimizer=sgd_optimizer,  loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [44]:
model.fit(X_train_std, y_train_cat, 
          validation_data=(X_test_std, y_test_cat), 
          epochs=10, batch_size=32)

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 1s 104us/sample - loss: 0.4761 - accuracy: 0.7941 - val_loss: 0.4554 - val_accuracy: 0.8050
Epoch 2/10
8000/8000 [==============================] - 0s 56us/sample - loss: 0.4651 - accuracy: 0.7941 - val_loss: 0.4446 - val_accuracy: 0.8050
Epoch 3/10
8000/8000 [==============================] - 0s 57us/sample - loss: 0.4553 - accuracy: 0.7952 - val_loss: 0.4350 - val_accuracy: 0.8040
Epoch 4/10
8000/8000 [==============================] - 0s 57us/sample - loss: 0.4489 - accuracy: 0.7944 - val_loss: 0.4292 - val_accuracy: 0.8050
Epoch 5/10
8000/8000 [==============================] - 0s 56us/sample - loss: 0.4430 - accuracy: 0.7951 - val_loss: 0.4249 - val_accuracy: 0.8075
Epoch 6/10
8000/8000 [==============================] - 0s 54us/sample - loss: 0.4411 - accuracy: 0.7971 - val_loss: 0.4229 - val_accuracy: 0.8105
Epoch 7/10
8000/8000 [==============================] - 0s 54us/sampl

In [0]:
# Accuracy has increased

In [0]:
# Building another model with input activation function as 'relu' and loss function as 'binary_crossentropy' as we have two output labels

In [0]:
#Clear out tensorflow memory
tf.keras.backend.clear_session()

#Initialize Sequential model
model1 = tf.keras.models.Sequential()

#Normalize the data
#model1.add(tf.keras.layers.BatchNormalization())

#Input Layer
model1.add(tf.keras.layers.Dense(20, activation='relu',input_shape=(10,)))

#Add 2nd hidden layer
model.add(tf.keras.layers.Dense(10, activation='relu')) 

#Add Dense Layer which provides 1 Output after applying sigmoid (Output Layer)
model1.add(tf.keras.layers.Dense(2, activation='softmax'))



In [0]:
#Comile the model
model1.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

In [49]:
model1.fit(X_train_std, y_train_cat, 
          validation_data=(X_test_std, y_test_cat), 
          epochs=10,
          batch_size=10)

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 2s 211us/sample - loss: 0.4888 - accuracy: 0.7811 - val_loss: 0.4306 - val_accuracy: 0.8190
Epoch 2/10
8000/8000 [==============================] - 1s 162us/sample - loss: 0.4345 - accuracy: 0.8111 - val_loss: 0.4117 - val_accuracy: 0.8300
Epoch 3/10
8000/8000 [==============================] - 1s 158us/sample - loss: 0.4200 - accuracy: 0.8219 - val_loss: 0.4028 - val_accuracy: 0.8305
Epoch 4/10
8000/8000 [==============================] - 1s 158us/sample - loss: 0.4079 - accuracy: 0.8280 - val_loss: 0.3896 - val_accuracy: 0.8400
Epoch 5/10
8000/8000 [==============================] - 1s 157us/sample - loss: 0.3963 - accuracy: 0.8339 - val_loss: 0.3771 - val_accuracy: 0.8525
Epoch 6/10
8000/8000 [==============================] - 1s 157us/sample - loss: 0.3853 - accuracy: 0.8406 - val_loss: 0.3684 - val_accuracy: 0.8535
Epoch 7/10
8000/8000 [==============================] - 1s 153us

In [0]:
# Compiling the above model with 'adam' as the optimizer

In [0]:
#Comile the model
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [52]:
model1.fit(X_train_std, y_train_cat, 
          validation_data=(X_test_std, y_test_cat), 
          epochs=10,
          batch_size=10)

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 2s 203us/sample - loss: 0.3586 - accuracy: 0.8515 - val_loss: 0.3427 - val_accuracy: 0.8695
Epoch 2/10
8000/8000 [==============================] - 1s 157us/sample - loss: 0.3551 - accuracy: 0.8537 - val_loss: 0.3402 - val_accuracy: 0.8705
Epoch 3/10
8000/8000 [==============================] - 1s 157us/sample - loss: 0.3532 - accuracy: 0.8541 - val_loss: 0.3373 - val_accuracy: 0.8710
Epoch 4/10
8000/8000 [==============================] - 1s 157us/sample - loss: 0.3514 - accuracy: 0.8528 - val_loss: 0.3353 - val_accuracy: 0.8725
Epoch 5/10
8000/8000 [==============================] - 1s 161us/sample - loss: 0.3502 - accuracy: 0.8555 - val_loss: 0.3354 - val_accuracy: 0.8700
Epoch 6/10
8000/8000 [==============================] - 1s 158us/sample - loss: 0.3493 - accuracy: 0.8556 - val_loss: 0.3387 - val_accuracy: 0.8695
Epoch 7/10
8000/8000 [==============================] - 1s 153us

In [0]:
# accuracy has further improved

9. Predict the results using 0.5 as a threshold (5 points)

In [54]:
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                220       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 42        
Total params: 262
Trainable params: 262
Non-trainable params: 0
_________________________________________________________________


In [55]:
#Predicting the Test set results
y_pred = model1.predict(X_test_std)
#Threshold of 50%
y_pred = (y_pred > 0.5)
y_pred[:5]

array([[ True, False],
       [ True, False],
       [False,  True],
       [ True, False],
       [ True, False]])

10. Print the Accuracy score and confusion matrix (2.5 points)

In [0]:
from sklearn.metrics import accuracy_score
acc=accuracy_score(y_test_cat, y_pred)

In [57]:
print('accuracy=',acc*100)

accuracy= 86.85000000000001


In [0]:
from sklearn import metrics

In [59]:
print ("Confusion Matrix for predictions with specified threshold 0.5")
pd.DataFrame(metrics.confusion_matrix(y_test_cat.argmax(axis=1), y_pred.argmax(axis=1)),
                 columns=['pred_neg', 'pred_pos'], index=['neg', 'pos'])

Confusion Matrix for predictions with specified threshold 0.5


,pred_neg,pred_pos
neg,1547,63
pos,200,190
